In [12]:
from dnn import pipeline
import os
import tensorflow as tf
from tensorflow.contrib.data import shuffle_and_repeat, batch_and_drop_remainder

In [15]:
meta_json = "ex.json"
hdf_fields = ['Cloud_Optical_Thickness', 'Cloud_Effective_Radius']
data_files = ["ex.tfrecord"]
shape = (64, 64)
batch_size = 32

In [21]:
chans, parser = pipeline.hdf_tfr_fn(hdf_fields, meta_json)
dataset = (
    tf.data.Dataset.from_tensor_slices(data_files)
    .apply(shuffle_and_repeat(100))
    .flat_map(tf.data.TFRecordDataset)
    .map(parser)
    .interleave(pipeline.patchify_fn(*shape[:2], chans), cycle_length=4)
    .shuffle(10000)
    .apply(batch_and_drop_remainder(batch_size))
)

In [22]:
x = dataset.make_one_shot_iterator().get_next()
with tf.Session() as sess:
    y = sess.run(x)

In [23]:
y.shape

(32, 64, 64, 2)